In [13]:
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

In [14]:
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  AIMessagePromptTemplate,
  HumanMessagePromptTemplate,
)

In [15]:
import pandas as pd

In [16]:
class MyChat:

    def __init__(self,model:str,template:str):
        self.vector_store = None
        self.retriever = None
        self.chain = None
        self.model = ChatOllama(model=model)
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)

    def ingest(self, pdf_file_path: str):
        docs = PyPDFLoader(file_path=pdf_file_path).load()
        chunks = self.text_splitter.split_documents(docs)
        return filter_complex_metadata(chunks)

    def make_vector_store(self, pdf_file_path:str):
        chunks = self.ingest(pdf_file_path)
        embedding_fn = FastEmbedEmbeddings()

        self.vector_store = Chroma.from_documents(documents=chunks, embedding=embedding_fn)

    def make_chain(self,template):
        self.retriever = self.vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 3,
                "score_threshold": 0.5,
            },
        )

        sys_message_prompt= SystemMessagePromptTemplate.from_template(template)
        example_human_history = HumanMessagePromptTemplate.from_template("Olá!")
        example_ai_history = AIMessagePromptTemplate.from_template("Oi, como você está hoje?")

        human_template="{input}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([sys_message_prompt, example_human_history, example_ai_history, human_message_prompt])

        self.chain = chat_prompt | self.model

    def ask(self, query: str,emendas:str):
        return self.chain.invoke({"context": self.retriever,"emendas":emendas,"input":query})

    def clear(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None

In [17]:
template="""Tarefa: Agrupamento Semântico de Emendas Parlamentares. Responda em português.
Você foi designado para realizar um agrupamento semântico das emendas parlamentares. Cada emenda é identificada por um texto descritivo.
Todas as emendas são referentes ao mesmo projeto de lei, PL.

Texto do PL:
<context>
{context}
</context>

Detalhes da Tarefa:
Você receberá um conjunto de emendas parlamentares, cada uma representada por um texto descritivo. As emendas podem abordar uma variedade de tópicos.
Seu modelo deve atribuir cada emenda a um grupo semântico com base em seus tópicos principais.
Certifique-se de que cada emenda seja atribuída a um único grupo e que todas as emendas sejam atribuídas a um grupo.

Exemplo de Emenda:
AQUI SE INICIA A EMENDA ID 000001 TEXTO

Texto das Emendas:
{emendas}

Question: {input}"""

In [18]:
query=f"""Realize o agrupamento semântico das emendas parlamentares fornecidas, atribuindo cada emenda a um grupo com base em seus tópicos principais."""

In [19]:
emendas_df = pd.read_csv("emendas-786-2020.csv")
emendas_df.shape

(8, 21)

In [20]:
inteiro_teor_todas_emendas = mensagem = "".join([f"AQUI SE INICIA A EMENDA ID {numero_emenda}:\n{text_proposto_emenda}\n" for numero_emenda, text_proposto_emenda in zip(emendas_df['NUMEROEMENDA'], emendas_df['TEXTOPROPOSTOEMENDA'])])
inteiro_teor_todas_emendas

'AQUI SE INICIA A EMENDA ID 30870016:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 32280017:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 36110022:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 39160035:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 39540014:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 39840011:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 40700034:\nSuprima-se o texto atual.\nAQUI SE INICIA A EMENDA ID 41300028:\nSuprima-se o texto atual.\n'

In [21]:
emendas = inteiro_teor_todas_emendas

In [24]:
estimativa_tokens = len(inteiro_teor_todas_emendas.split())
estimativa_tokens

88

### PL-786-2020

In [25]:
chat = MyChat("llama3",template)

chat.make_vector_store("PL-786-2020.pdf")

chat.make_chain(template)

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 57808.17it/s]


In [26]:
response = chat.ask(query,emendas)
print(response.content)

Vou realizar o agrupamento semântico das emendas parlamentares!

Aparentemente, todas as emendas referem-se à supressão de texto atual. Com isso, vou atribuir cada emenda a um único grupo com base nesse tópico principal.

**Grupo 1: Supressão de texto atual**

* Emenda ID 3087: "Suprima-se o texto atual."
* Emenda ID 3228: "Suprima-se o texto atual."
* Emenda ID 3611: "Suprima-se o texto atual."
* Emenda ID 3916: "Suprima-se o texto atual."
* Emenda ID 3954: "Suprima-se o texto atual."
* Emenda ID 3984: "Suprima-se o texto atual."
* Emenda ID 4070: "Suprima-se o texto atual."
* Emenda ID 4130: "Suprima-se o texto atual."

Espero que isso seja o agrupamento semântico correto!


### PL-20-2020

In [21]:
emendas_df = pd.read_csv("emendas-20-2020.csv")
emendas_df.shape

(77, 21)

In [22]:
inteiro_teor_todas_emendas = "".join([f"AQUI SE INICIA A EMENDA ID {numero_emenda}:\n{text_proposto_emenda}\n" for numero_emenda, text_proposto_emenda in zip(emendas_df['NUMEROEMENDA'], emendas_df['TEXTOPROPOSTOEMENDA'])])
emendas = inteiro_teor_todas_emendas

In [23]:
estimativa_tokens = len(inteiro_teor_todas_emendas.split())
estimativa_tokens

4160

In [24]:
chat = MyChat("llama3",template)

chat.make_vector_store("PL-20-2020.pdf")

chat.make_chain(template)

/home/pedrest/Documentos/TCC/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 3.55MB/s]
config.json: 100%|██████████| 706/706 [00:00<00:00, 1.81MB/s]t]
ort_config.json: 100%|██████████| 1.27k/1.27k [00:00<00:00, 7.30MB/s]
tokenizer_config.json: 100%|██████████| 1.24k/1.24k [00:00<00:00, 6.50MB/s]
special_tokens_map.json: 100%|██████████| 695/695 [00:00<00:00, 2.60MB/s]

README.md: 100%|██████████| 28.0/28.0 [00:00<00:00, 33.9kB/s]
Fetching 9 files:  22%|██▏       | 2/9 [00:01<00:04,  1.47it/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.40MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 747kB/s]







model_optimized.onnx: 100%|██████████| 66.5M/66.5M [00:07<00:00, 8.65MB/s]
Fetching 9 files: 100%|██████████| 9/9 [0

In [25]:
response = chat.ask(query,emendas)
print(response)

content=' Based on the given emendas, I have grouped them into the following categories based on their primary topics:\n\n1. Budget and Priorities (overall)\n   - Emenda 50360002, 50360003, 50370011, 60000003, 60000030: These emendas mention the overall budget and priorities for the federal administration in 2020.\n   - Emenda 71250006, 71250012: These emendas specify some of the priorities, such as education, public security, and the enforcement of laws against violence against women.\n\n2. Education (PNE)\n   - Emenda 30090014: This emenda refers specifically to the National Education Plan (PNE).\n   - Emenda 60000031, 71080009: These emendas mention the importance of fulfilling the goals set in the National Education Plan.\n\n3. Enforcement of laws against violence against women\n   - Emenda 50360004, 50360009: These emendas include actions related to the enforcement of laws against violence against women as a priority for the federal administration in 2020.\n\n4. Miscellaneous\n   